In [1]:
!pip install transformers
!pip install sentencepiece
# !pip install swifter

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import torch
import pandas as pd
# import swifter

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
qa_tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [4]:
prop_tokenizer = AutoTokenizer.from_pretrained("chentong00/propositionizer-wiki-flan-t5-large")
prop_model = AutoModelForSeq2SeqLM.from_pretrained("chentong00/propositionizer-wiki-flan-t5-large").to(device)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [5]:
def extract_propositions(sentence):
    title = ""
    section = ""
    content = sentence

    input_text = f"Title: {title}. Section: {section}. Content: {content}"

    input_ids = prop_tokenizer(input_text, return_tensors="pt").input_ids
    input_ids.to(device)
    outputs = prop_model.generate(input_ids.to(device), max_new_tokens=512).cpu()

    output_text = prop_tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        prop_list = json.loads(output_text)
    except:
        prop_list = []
        print("[ERROR] Failed to parse output text as JSON.")
        print(output_text)
        return prop_list

    return prop_list

In [6]:
text = 'Verbalisation: The percentage of anaemia that is iron deficiency anaemia ranges from 11% in Mtwara to 56% in Arusha.'
text2 = 'Nigeria is in the middle, with about 1/3 of women in each empowerment group (Figure 1).'
text3 = 'Mutlivariable models show that in 5 countries, women with medium and high empowerment have significantly lower ideal numbers of children than women with low empowerment (Figure 3).'
text4 = 'Even when controlling for background characteristics, medium and highly empowered young women in Nepal, Nigeria, Senegal, Uganda, and Zambia want fewer children than their less empowered peers.'
text5 = 'In Nigeria, young women with high empowerment are much more likely to be currently using contraception than those with low empowerment, and among non-users, those with high empowerment are much more likely to intend to use in the future than those with low empowerment.'
text6 = 'Over half of women in Manyara (58%), Mara (56%), and Dodoma (52%) reported at least one serious problem in accessing health care for themselves.'
text7 = 'Women living in the regions of Zanzibar are least likely to report problems accessing health care for themselves.'
text8 = 'Adolescent childbearing ranges from a low of 4% of women age 15-19 in Pemba North and Town West to a high of 46% in Lindi'
text9 = 'In contrast, more than one-third (34%) of women in Unguja South took iron supplements for 90 or more days.'
text10 = 'Access to improved water varies by region, from 67% in Khatlon to 99% in Dushanbe.'

props = extract_propositions(text10)
print(props)

['Access to improved water varies by region.', 'In Khatlon, 67% of people have access to improved water.', 'In Dushanbe, 99% of people have access to improved water.']


In [7]:
def generate_qa_pairs(context, num_pairs):
  inputs = qa_tokenizer(context, return_tensors="pt")
  inputs.to(device)
  qa_pairs = []

  for i in range(num_pairs):
    outputs = qa_model.generate(**inputs, max_length=100, do_sample=True)
    question_answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=False)
    question_answer = question_answer.replace(qa_tokenizer.pad_token, "").replace(qa_tokenizer.eos_token, "")
    try:
      question, answer = question_answer.split(qa_tokenizer.sep_token)

      qa_pair = {"question": question.lstrip(' ').rstrip(' '), "answer": answer.lstrip(' ').rstrip(' ') + "."}
      qa_pairs.append(qa_pair)
    except:
      print("FAILED TO SEPERATE QA")
      print(question_answer)
      print(context)

  return qa_pairs

In [8]:
pairs = []

for prop in props:
  pair = generate_qa_pairs(prop, 10)
  print(pair)
  print()
  pairs.append(pair)

pairs

[{'question': 'Who has their own access to improved water?', 'answer': 'each region.'}, {'question': "What is one way that people get water that's improved?", 'answer': 'Access to improved water.'}, {'question': 'What is a major factor in whether or not a area has access to improved water?', 'answer': 'region.'}, {'question': 'Where can one find improved access to water?', 'answer': 'region.'}, {'question': 'What is a geographic variation of access to water?', 'answer': 'regions.'}, {'question': 'What do regions of America have to offer in terms of better water?', 'answer': 'improved water.'}, {'question': 'What is the range of public access?', 'answer': 'access to improved water varies by region..'}, {'question': 'Access to improved water is not the same across each region.', 'answer': 'Access to improved water varies by region.'}, {'question': 'What is one major difficulty of obtaining improved water?', 'answer': 'Access.'}, {'question': 'What is the local standard of access for impr

[[{'question': 'Who has their own access to improved water?',
   'answer': 'each region.'},
  {'question': "What is one way that people get water that's improved?",
   'answer': 'Access to improved water.'},
  {'question': 'What is a major factor in whether or not a area has access to improved water?',
   'answer': 'region.'},
  {'question': 'Where can one find improved access to water?',
   'answer': 'region.'},
  {'question': 'What is a geographic variation of access to water?',
   'answer': 'regions.'},
  {'question': 'What do regions of America have to offer in terms of better water?',
   'answer': 'improved water.'},
  {'question': 'What is the range of public access?',
   'answer': 'access to improved water varies by region..'},
  {'question': 'Access to improved water is not the same across each region.',
   'answer': 'Access to improved water varies by region.'},
  {'question': 'What is one major difficulty of obtaining improved water?',
   'answer': 'Access.'},
  {'question': 

In [ ]:
def rightmost_qa_pairs(question_answer_pairs, proposition):
  proposition = proposition.rstrip('.')

  filtered_pairs = []

  filtered_pair = question_answer_pairs[0]
  max_length = 0

  for pair in question_answer_pairs:
      question, answer = pair["question"], pair["answer"]

      if proposition.endswith(answer):
          filtered_pairs.append(pair)

  return filtered_pairs if len(filtered_pairs) != 0 else question_answer_pairs

In [9]:
def filter_qa(data):
  filtered_data = []

  for item in data:
    if item["question"].endswith("?") and item["answer"] not in [".", ""]:
      filtered_data.append(item)

  return filtered_data

In [10]:
def filter_qa_answer_in_question(data):
  filtered_data = []

  for item in data:
    if item["answer"].lower().rstrip('.') not in item["question"].lower():
      filtered_data.append(item)

  return filtered_data

In [11]:
def filter_ans_duplicates(data, text):
  filtered_data = []
  qa_map = {}  # Map to track questions or answers and their associated items

  for item in data:
    qa_key = item["answer"]

    if qa_key not in qa_map:
      # No duplicates encountered yet
      qa_map[qa_key] = item
    else:
      # Duplicate exists, calculate similarity and keep the more similar one
      existing_item = qa_map[qa_key]
      new_similarity = calculate_similarity(item["question"], text)
      existing_similarity = calculate_similarity(existing_item["question"], text)

      if new_similarity > existing_similarity:
        qa_map[qa_key] = item

  # Add unique items from the map to filtered_data
  filtered_data = list(qa_map.values())
  return filtered_data

def calculate_similarity(text1, text2):
  text1_words = set(text1.lower().split())
  text2_words = set(text2.lower().split())

  # Calculate precision, recall, and F1 score
  intersection = len(text1_words.intersection(text2_words))
  precision = intersection / len(text1_words) if len(text1_words) > 0 else 0
  recall = intersection / len(text2_words) if len(text2_words) > 0 else 0
  f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

  return f1

In [12]:
def filter_qa_by_similarity(data, text):
  filtered_data = []
  max_similarity = 0
  max_index = None

  # Calculate similarity for each question
  for i, item in enumerate(data):
    question_words = set(item["question"].lower().split()) | set(item["answer"].lower().split())
    text_words = set(text.lower().split())
    similarity = len(question_words.intersection(text_words)) / len(question_words.union(text_words))
    item["similarity"] = similarity

    if similarity > max_similarity:
      max_similarity = similarity
      max_index = i

  # Filter questions within 20% of the top similarity
  threshold = max_similarity * 0.8
  for item in data:
    if item["similarity"] >= threshold:
      filtered_data.append(item)

  # Calculate similarity scores as percentages
  for item in filtered_data:
    item["similarity_score"] = round(item["similarity"] * 100, 2)

  # Sort by similarity score in descending order
  filtered_data.sort(key=lambda item: item["similarity_score"], reverse=True)

  # Keep only the top 3 pairs
  if (len(filtered_data) > 3):
    filtered_data = filtered_data[:3]

  for item in filtered_data:
    del item["similarity"]
    del item["similarity_score"]

  return filtered_data

In [13]:
import re

def filter_qa_by_number(data, text):
  filtered_data = []
  number_regex = re.compile(r"\d+(\.\d+)?%?\.?")

  for item in data:
      nums = extract_number(item['question'] + item['answer'])
      if all([num in text for num in nums]):
        filtered_data.append(item)

  return filtered_data


def extract_number(text):
  number_regex = r"[-+]?\d*\.?\d+"  # Match integers, floats, with optional sign
  return re.findall(number_regex, text)

In [15]:
new_pairs = [filter_qa(pair) for pair in pairs]
new_pairs = [filter_qa_by_number(pair, text10) for pair in new_pairs]
new_pairs = [filter_qa_answer_in_question(pair) for pair in new_pairs]
new_pairs = [filter_ans_duplicates(pair, text10) for pair in new_pairs]
new_pairs = [filter_qa_by_similarity(pair, text10) for pair in new_pairs]
new_pairs

[[{'question': 'What is the range of public access?',
   'answer': 'access to improved water varies by region..'},
  {'question': 'What is the local standard of access for improved water?',
   'answer': 'Access to improved water varies by region.'}],
 [{'question': 'What percentage of people in Khatlon have access to improved water?',
   'answer': '67%.'},
  {'question': 'Why doesn’t everyone have water there?',
   'answer': 'access to improved water.'}],
 [{'question': 'What percentage of people have access to improved water to Dushanbe?',
   'answer': '99%.'},
  {'question': 'On average, in Dushanbe, what percent of people have access to improved water?',
   'answer': '99.'}]]

In [16]:
def most_similar_pair(pairs, sentence):
  # question_list = [entry["question"] for entry in sentence_list]

  # Tokenize input sentences
  tokenized_input = set(sentence.lower().split())

  # Initialize variables to track maximum overlap and corresponding sentence
  max_overlap = 0
  most_overlapping_sentence = {}

  for pair in pairs:
    candidate_sentence = pair['question']
    # Tokenize candidate sentence
    tokenized_candidate = set(candidate_sentence.lower().split())

    # Calculate word overlap
    overlap_count = len(tokenized_input.intersection(tokenized_candidate))

    # Update maximum overlap and corresponding sentence if needed
    if overlap_count > max_overlap:
        max_overlap = overlap_count
        most_overlapping_sentence = pair

  return most_overlapping_sentence

In [19]:
most_similar_pair(pairs[2], text10)

{'question': 'What percentage of people have access to improved water to Dushanbe?',
 'answer': '99%.'}

In [ ]:
def concat_qa_pairs(qa_pairs):
  try:
    joined_qa_pairs = " ".join(f"{qa_pair['answer']} {qa_pair['question']} |" for qa_pair in qa_pairs)
    # return "Blueprint: " + joined_qa_pairs + " Summary: "
    return joined_qa_pairs
  except:
    print("no blueprint could be made")

In [ ]:
def pipeline(reference):
  # print("Reference:")
  # print(reference)
  # print()
  propositions = extract_propositions(reference)

  all_qa_pairs = []

  for proposition in propositions:
    # print(proposition)
    # print()
    qa_pairs = generate_qa_pairs(proposition, 5)
    # print(qa_pairs)
    # print()
    # right_pairs = rightmost_qa_pairs(qa_pairs, proposition)
    pair = most_similar_pair(qa_pairs, proposition)
    # print(pair)
    # print()
    all_qa_pairs.append(pair)

  return concat_qa_pairs(all_qa_pairs)

In [ ]:
def new_pipeline(reference):
  propositions = extract_propositions(reference)

  all_qa_pairs = []

  for proposition in propositions:
    qa_pairs = generate_qa_pairs(proposition, 5)
    new_pairs = filter_qa(qa_pairs)
    new_pairs = filter_qa_by_number(qa_pairs, reference)
    new_pairs = filter_qa_answer_in_question(qa_pairs)
    new_pairs = filter_ans_duplicates(qa_pairs, reference)
    new_pairs = filter_qa_by_similarity(qa_pairs, reference)

    all_qa_pairs.append(new_pairs)

  return concat_qa_pairs(flatten_list(all_qa_pairs))

def flatten_list(nested_list):
  flat_list = [item for sublist in nested_list for item in sublist]
  return flat_list

In [ ]:
new_pipeline(text4)

'fewer. When controlling for backgrounds, how many children do medium and highly empowered young women want compared to their less empowered peers? | medium and highly empowered. What types of young women want more children than less empowered peers? |'

In [ ]:
pipeline(text4)

'children. What do most of the high educated and highly empowered young women want fewer to be? |'

In [ ]:
import ast

# df = pd.read_csv('train_skip_no_refs_eng_blueprints.csv', delimiter="\t")
df = pd.read_csv('dev.csv', delimiter="\t")
# df['table_text'] = df['table_text'].apply(ast.literal_eval)

In [ ]:
def process_train_dataframe(df):
    df['blueprint'] = ''
    print(len(df))
    print()

    for index, row in df.iterrows():
        print(index)

        example_id = row['example_id']
        table_text = row['table_text']

        # print(table_text)

        if '-en-' in example_id:
            df.at[index, 'blueprint'] = []  # Use df.at to modify the original DataFrame

            for i in range(0, len(row['table_text'])):
                # print("Table text:")
                # print(row['table_text'])
                # print()
                blueprint = pipeline(row['table_text'][i])
                df.at[index, 'blueprint'].append(blueprint)

    return df

In [ ]:
def process_dev_dataframe(df):
    df['blueprint'] = ''
    print(len(df))
    print()

    for index, row in df.iterrows():
        print(index)

        example_id = row['example_id']
        table_text = row['target']

        # print(table_text)

        if '-en-' in example_id:
            df.at[index, 'blueprint'] = []  # Use df.at to modify the original DataFrame

            blueprint = pipeline(table_text)
            df.at[index, 'blueprint'] = blueprint

    return df

In [ ]:
df = process_dev_dataframe(df)

754

0
1
FAILED TO SEPERATE QA
 There is significant diversity in the family formation process across countries
There is significant diversity in the family formation process across countries.
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
FAILED TO SEPERATE QA
 Nairobi's foetility rank is the lowest in the province
Fertility is lowest in Nairobi province.
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
FAILED TO SEPERATE QA
 Figure 9.1 is intended to assess changes in coverage indicators.
The purpose of Figure 9.1 is to assess the magnitude of recent changes in key coverage indicators.
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
FAILED TO SEPERATE QA
 Seve

In [ ]:
df.to_csv('dev_eng_blueprints_only_eng.csv', sep='\t', index=False)